In [1]:
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
from urllib.error import URLError, HTTPError

def trata_html(html):
    return ' '.join(html.split()).replace("> <", "><");

url = 'https://www.olx.com.br/imoveis/aluguel/apartamentos/estado-pb/paraiba/joao-pessoa'
cabecalho = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'}

try:
    request = Request(url, headers = cabecalho)
    response = urlopen(request)
except HTTPError as erro:
    print(erro.status, erro.reason)
except URLError as erro:
    print(erro.reason)
    
html = response.read().decode('utf-8')
html = trata_html(html)

In [2]:
soup = BeautifulSoup(html, 'html.parser')
print(soup.prettify())

<html data-reactroot="" lang="pt-BR">
 <head>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <meta content="abuy-list-item-tags-grid_enabled.adv-active-insurads_enabled.adv-gemini-less-positions_enabled.cdrelrank-synonym_on.dispute-automatic-decision_A.listing-transactional-badge_enabled.newadcard_enabled.re-attributes-bjQTZ-bjfilters_control.search-intention-filters_control.trackexp-links-azuis_control" name="x-testab-debugger"/>
  <meta content="#6e0ad6" name="theme-color"/>
  <meta content="yes" name="apple-mobile-web-app-capable"/>
  <meta content="black-translucent" name="apple-mobile-web-app-status-bar-style"/>
  <link href="https://olx.com.br/busca?ca=83_s&amp;cg=1002&amp;ret=1020&amp;w=5&amp;zn=4794" media="only screen and (max-width: 640px)" rel="alternate"/>
  <meta charset="utf-8"/>
  <meta content="uqJwtiNExrXVCtT-w4AryaEhN-mU4ognZGnDIX3ds-w" name="google-site-verification"/>
  <meta content="telephone=no" name="format-detection"/>
  <meta content

In [3]:
cards = []
card = {}

In [4]:
anuncio = soup.find('div', {'class': 'sc-fgfRvd fZoIzL'})
anuncio

<div class="sc-fgfRvd fZoIzL"><div class="sc-hIVACf dUnJVd"><div class="sc-gpHHfC kJrkKt"><h3 aria-label="categoria do anúncio: Apartamento para alugar">Apartamento para alugar</h3><h2 aria-label="título do anúncio: Apartamento (aluguel)">Apartamento (aluguel)</h2><div aria-label="detalhes do anúncio" class="sc-gVyKpa efbmpH" data-testid="labelGroup"><span aria-label="2 quartos"><svg color="currentColor" height="16" size="16" viewbox="0 0 32 32" width="16" xmlns="http://www.w3.org/2000/svg"><path d="M26.978 13.293V5.717c0-.104.032-.2.086-.28a.498.498 0 01-.094-.29c0-.715-.629-1.314-1.405-1.314H6.427c-.777 0-1.405.598-1.405 1.313a.498.498 0 01-.09.286c.057.08.09.179.09.285v7.655c.475-.067.968-.13 1.473-.187a.505.505 0 01-.003-.057V11.96c0-1.38 1.177-2.502 2.602-2.502h4.767c.95 0 1.79.499 2.245 1.24a2.633 2.633 0 012.244-1.24h4.766c1.426 0 2.602 1.12 2.602 2.502v1.151c.446.056.867.117 1.26.183zm.998.192a.5.5 0 01.35.344l2.433 8.436c.04.141.017.281-.05.394a.5.5 0 01.027.165v5.986a.5.5 0 0

In [5]:
card["Valor"] = anuncio.find('span', {'class': "main-price sc-ifAKCX bytyxL"}).getText()
card

{'Valor': 'R$ 700'}

In [6]:
itens = anuncio.find('div', {'class': "sc-gVyKpa efbmpH"}).findAll("span")
for item in itens:
    card[item.get("aria-label").split()[1]] = item.get("aria-label").split()[0]
card

{'Valor': 'R$ 700',
 'quartos': '2',
 'metros': '60',
 'vaga': '1',
 'banheiros': '2'}

In [7]:
anuncio.find('div', {'class': 'sc-hjRWVT lgwjMd'}).find('span').getText().split(', ')[-1]

'Mangabeira'

In [8]:
anuncios = soup.find('ul', {'class': 'sc-1mburcf-0 todbs'}).findAll('li', class_ = "sc-1mburcf-1 dzAaoj")
len(anuncios)
anuncios[0].getText()

'6Apartamento para alugarApartamento (aluguel)260m²12R$ 700João Pessoa, Mangabeira | Hoje, 11:19'

In [12]:
cards = []
for anuncio in anuncios:
    card = {}
    card['Bairro'] = anuncio.find('div', {'class': 'sc-hjRWVT lgwjMd'}).find('span').getText().split(', ')[-1]
    card["Valor"] = anuncio.find('span', {'class': "main-price sc-ifAKCX bytyxL"}).getText()
    itens = anuncio.find('div', {'class': "sc-gVyKpa efbmpH"}).findAll("span")
    for item in itens:
        card[item.get("aria-label").split()[1]] = item.get("aria-label").split()[0]
    cards.append(card)
len(cards)

50

In [51]:
import pandas as pd
df = pd.DataFrame(cards)
df.head()

,Bairro,Valor,quartos,metros,vaga,banheiros,quarto,banheiro,vagas,ou
0,Mangabeira,R$ 700,2,60,1,2,NaN,NaN,NaN,NaN
1,Bessa,R$ 1.900,NaN,35,1,NaN,1,1,NaN,NaN
2,Manaíra,R$ 3.000,3,130,NaN,4,NaN,NaN,2,NaN
3,Jardim Cidade Universitária,R$ 700,2,70,1,2,NaN,NaN,NaN,NaN
4,Cabo Branco,R$ 4.200,2,62,1,2,NaN,NaN,NaN,NaN


In [52]:
df['vagas'].fillna(df['vaga'], inplace=True)
del df['vaga']
df['vagas'].fillna(0, inplace=True)

In [53]:
df['quartos'].fillna(df['quarto'], inplace=True)
del df['quarto']

In [54]:
df['banheiros'].fillna(df['ou'], inplace=True)
df['banheiros'].fillna(df['banheiro'], inplace=True)
del df['ou'], df['banheiro']

In [55]:
df.rename(columns = {'quartos': 'Quartos', 'metros': "Área", "banheiros": "Banheiros", "vagas": "Vagas"}, inplace=True)
df.head()

,Bairro,Valor,Quartos,Área,Banheiros,Vagas
0,Mangabeira,R$ 700,2,60,2,1
1,Bessa,R$ 1.900,1,35,1,1
2,Manaíra,R$ 3.000,3,130,4,2
3,Jardim Cidade Universitária,R$ 700,2,70,2,1
4,Cabo Branco,R$ 4.200,2,62,2,1


In [59]:
df.to_csv(r'C:\Users\Lenovo\OneDrive\Documentos\Projects\Dados\aluguel.csv', sep=';', index=False, encoding='utf-8-sig')